In [ ]:
import numpy as np
import os, json
import re, subprocess
import meshio

def extract_floats(text: str):
    return list(map(float, re.findall(r'[-+]?\ *[0-9]+\.?[0-9]*(?:[Ee]\ *[-+]?\ *[0-9]+)?', text)))

def find_end(str, keyword):
    return str.find(keyword) + len(keyword)

polyfem_bin = "~/adjoint/build/release/PolyFEM_bin"

# from Github repo https://github.com/MmgTools/mmg
mmg_bin = "~/mmg/build/bin/mmg3d_O3"

mesh_path = "beam_filled_10k_scaled.msh"
for iter in range(100):

    # optimization configuration
    out_dir = "./opt_" + str(iter)
    log_path = os.path.join(out_dir, "log")
    json_path = "opt-" + str(iter) + ".json"
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

        with open("opt.json", 'r') as f:
            opt_args = json.load(f)
            opt_args["output"]["directory"] = out_dir
        
        with open(json_path, 'w') as f:
            json.dump(opt_args, f, indent=4)

        # simulation configuration
        with open("run.json", 'r') as f:
            state_args = json.load(f)
            state_args["output"]["directory"] = "./opt_" + str(iter)
            state_args["geometry"][0]["mesh"] = mesh_path
        
        with open("run.json", 'w') as f:
            json.dump(state_args, f, indent=4)
        
        # run shape optimization
        # if iter > 0:
        if os.system(polyfem_bin + " -j " + json_path + " --ns > " + log_path) != 0:
            with open(log_path, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    # adjust the number of variables in shape optimization due to remesh
                    if 'Indexing size and output size of the Parametrization do not match!' in line:
                        print("Wrong number of variables!")
                        ndofs = int(extract_floats(line[find_end(line, 'Indexing size and output size of the Parametrization do not match!'):])[0])
                        opt_args["parameters"][0]["number"] = ndofs
                        with open(json_path, 'w') as g:
                            json.dump(opt_args, g, indent=4)
                        os.system(polyfem_bin + " -j " + json_path + " --ns > " + log_path)
                        break

    # find last iteration
    with open(log_path, 'r') as f:
        lines = f.readlines()
        last_iter = -1
        for line in lines:
            if "Saving iteration" in line:
                last_iter = int(line[find_end(line, "Saving iteration"):-1])
            if "Save rest mesh to file" in line:
                opt_mesh_path = line[find_end(line, "Save rest mesh to file "):line.find(" ...")]
        print("Last iteration: ", last_iter)
        if last_iter <= 0:
            raise Exception("Failed to take a step in last round of optimization!")

    # if quality is bad, remesh
    mesh_path = os.path.join(out_dir, "optimized_remesh.msh")
    m = meshio.read(opt_mesh_path)
    m.write(opt_mesh_path, file_format='gmsh22')
    with open(os.path.join(out_dir, "remesh.log"), 'w') as fd:
        subprocess.run([mmg_bin, 
                        "-in", opt_mesh_path, 
                        "-nosurf", "-optim", 
                        "-out", mesh_path], stdout=fd)